# Roku Experiments

## Resources
  - https://github.com/flirc/sdk/blob/master/cli/src/cmds/ir_transmit.c
  - https://github.com/hulu/roku-dev-cli/blob/master/roku_dev_cli/roku_dev_cli.py
  - https://gist.github.com/peterjc/7422624
  - https://www.exploitee.rs/index.php/Exploiting_Roku

In [21]:
import socket
import requests


def ping(server: str, port: int = 8060, timeout: int = 3):
  try:
    socket.setdefaulttimeout(timeout)
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.connect((server, port))
  except OSError as error:
    return False
  else:
    s.close()
    return True


def keypress(server: str, key: str, port: int = 8060):
  requests.post(f"http://{server}:{port}/keypress/{key}")


# TODO: Scan subnet
OUI = [
  "A8B57C", "BCD7D4", "20EFBD", "CC6DA0",
  "DC3A5E", "B83E59", "AC3A7A"
  ]


ROKU_IP = "192.168.1.210"
DEVICE_IP = socket.gethostbyname(socket.gethostname())
# DEVICE_IP = socket.gethostbyname(socket.getfqdn())
# DEVICE_IP = socket.gethostbyname_ex(socket.gethostname())[-1]

MACRO = []


if not ping(ROKU_IP): exit()

keypress(ROKU_IP, 'home')


In [ ]:
class Device:
  def __init__(self):
    ...

# import socket
# def get_ip():
#     s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
#     s.settimeout(0)
#     try:
#         # doesn't even have to be reachable
#         s.connect(('10.254.254.254', 1))
#         IP = s.getsockname()[0]
#     except Exception:
#         IP = '127.0.0.1'
#     finally:
#         s.close()
#     return IP
# print(get_ip())

In [19]:
# RAW ICMP

import socket
import struct
import time

def main():
    ping('192.168.1.1')

def ping(destination):
    sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM, socket.getprotobyname('icmp'))
    sock.settimeout(10.0)
    start_time = time.time_ns() # python 3.7+ only

    payload = struct.pack('L', start_time)
    sock.sendto(encode(payload), (destination, 0))
    while (time.time_ns() - start_time) // 1_000_000_000 < 10:
        try:
            data, source = sock.recvfrom(256)
        except socket.timeout:
            print('timed out')
            return
        message_type, message_code, check, identifier, sequence_number = struct.unpack('bbHHh', data[:8])
        if source == (destination, 0) and message_type == ICMP.ECHO_REPLY and data[8:] == payload:
            print((time.time_ns() - start_time) // 1_000_000, 'ms')
            break
        else:
            print('got unexpected packet from %s:' % source[0], message_type, data[8:])
    else:
        print('timed out')

def encode(payload: bytes):
    # calculate checksum with check set to 0
    checksum = calc_checksum(icmp_header(ICMP.ECHO_REQUEST, 0, 0, 1, 1) + payload)
    # craft the packet again with the checksum set
    return icmp_header(ICMP.ECHO_REQUEST, 0, checksum, 1, 1) + payload

def icmp_header(message_type, message_code, check, identifier, sequence_number) -> bytes:
    return struct.pack('bbHHh', message_type, message_code, check, identifier, sequence_number)

def calc_checksum(data: bytes) -> int:
    '''RFC 1071'''
    # code stolen from https://github.com/alessandromaggio/pythonping/blob/a59ce65a/pythonping/icmp.py#L8
    '''
    MIT License

    Copyright (c) 2018 Alessandro Maggio

    Permission is hereby granted, free of charge, to any person obtaining a copy
    of this software and associated documentation files (the "Software"), to deal
    in the Software without restriction, including without limitation the rights
    to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
    copies of the Software, and to permit persons to whom the Software is
    furnished to do so, subject to the following conditions:

    The above copyright notice and this permission notice shall be included in all
    copies or substantial portions of the Software.

    THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
    IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
    FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
    AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
    LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
    OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
    SOFTWARE.
    '''
    subtotal = 0
    for i in range(0, len(data)-1, 2):
        subtotal += (data[i] << 8) + data[i+1]
    if len(data) % 2:
        subtotal += (data[len(data)-1] << 8)
    while subtotal >> 16:
        subtotal = (subtotal & 0xFFFF) + (subtotal >> 16)
    check = ~subtotal
    return ((check << 8) & 0xFF00) | ((check >> 8) & 0x00FF)

class ICMP:
    ECHO_REPLY = 0
    ECHO_REQUEST = 8


main()

got unexpected packet from 192.168.1.1: 69 b'@\x01\xd2\xc4\xc0\xa8\x01\x01\xc0\xa8\x01\xa6\x00\x00\xa1\xfc\x01\x00\x01\x00P\x80\xbb\xc8C\xa3\x0c\x17'
timed out
